In [21]:
!git clone https://github.com/dml-qom/FarsTail.git


fatal: destination path 'FarsTail' already exists and is not an empty directory.


In [23]:
import pandas as pd
import numpy as np

In [22]:
train_data = pd.read_csv('/content/FarsTail/data/Train-word.csv', sep='\t')
val_data = pd.read_csv('/content/FarsTail/data/Val-word.csv', sep='\t')
test_data = pd.read_csv('/content/FarsTail/data/Test-word.csv', sep='\t')

In [18]:
!pip install transformers
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/config.json
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-fa-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_t

In [19]:
label_encoder = {'e':0,'c':1,'n':2}
y_train = [label_encoder[i] for i in train_data['label'].values.tolist()]
y_valid = [label_encoder[i] for i in val_data['label'].values.tolist()]
y_test = [label_encoder[i] for i in test_data['label'].values.tolist()]

In [20]:
def Tokenizer_persian(data):
  return tokenizer(data, padding=True, truncation=True, max_length=32)

train_p = Tokenizer_persian(train_data['premise'].values.tolist())
train_h = Tokenizer_persian(train_data['hypothesis'].values.tolist())

valid_p = Tokenizer_persian(val_data['premise'].values.tolist())
valid_h = Tokenizer_persian(val_data['hypothesis'].values.tolist())

test_p = Tokenizer_persian(test_data['premise'].values.tolist())
test_h = Tokenizer_persian(test_data['hypothesis'].values.tolist())


In [24]:
train = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
valid = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}
test = {'input_ids':[], 'token_type_ids':[], 'attention_mask':[]}

for i in range(len(train_p['input_ids'])):
  train['input_ids'].append(train_p['input_ids'][i]+train_h['input_ids'][i])
  train['token_type_ids'].append(train_p['token_type_ids'][i]+train_h['token_type_ids'][i])
  train['attention_mask'].append(train_p['attention_mask'][i]+train_h['attention_mask'][i])

for i in range(len(valid_p['input_ids'])):
  valid['input_ids'].append(valid_h['input_ids'][i]+valid_p['input_ids'][i])
  valid['token_type_ids'].append(valid_h['token_type_ids'][i]+valid_p['token_type_ids'][i])
  valid['attention_mask'].append(valid_h['attention_mask'][i]+valid_p['attention_mask'][i])


for i in range(len(test_h['input_ids'])):
  test['input_ids'].append(test_p['input_ids'][i]+test_h['input_ids'][i])
  test['token_type_ids'].append(test_p['token_type_ids'][i]+test_h['token_type_ids'][i])
  test['attention_mask'].append(test_p['attention_mask'][i]+test_h['attention_mask'][i])


In [25]:
from transformers import BertTokenizer, BertForSequenceClassification
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_si

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-uncased/snapshots/800c34f3d5aa174fe531f560b44b8d14592225b7/pytorch_model.bin
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly

In [ ]:
train.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [26]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(train, y_train)
valid_dataset = Dataset(valid, y_valid)
test_dataset = Dataset(test)

In [27]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [28]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3178 MB |    4485 MB |    8273 GB |    8270 GB |\n|       from large pool |    3176 MB |    4455 MB |    8056 GB |    8052 GB |\n|       from small pool |       2 MB |      53 MB |     217 GB |     217 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    3178 MB |    4485 MB |    8273 GB |    8270 GB |\n|       from large pool |    3176 MB |    4455 MB |

In [29]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7266
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2275
  Number of trainable parameters = 167358723


Step,Training Loss,Validation Loss
500,1.106400,1.099882
1000,1.107300,1.100431
1500,1.107600,1.100228
2000,1.099700,1.098453


***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1537
  Batch size = 16
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin


Training completed. Do not fo

TrainOutput(global_step=2275, training_loss=1.1045101626888736, metrics={'train_runtime': 551.0329, 'train_samples_per_second': 65.931, 'train_steps_per_second': 4.129, 'total_flos': 1194863808257280.0, 'train_loss': 1.1045101626888736, 'epoch': 5.0})

In [30]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM
trained_bert = BertForMaskedLM.from_pretrained('HooshvareLab/bert-fa-base-uncased')
tokenizer_trained_bert = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model.to('cuda')
trained_bert.to('cuda')
trained_bert.bert.encoder.layer=model.bert.encoder.layer

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/pytorch_model.bin
Generate config GenerationConfi

In [31]:
def predict_masked_word(sentence, modek,relationship=0):
    """ Predicts the masked word in a given sentence using a transformer model and it's tokenizer. """

    # Tokenize the input sentence  
    tokenized_sentence = tokenizer.tokenize("[CLS] " + sentence + "[SEP]")
    mask_index = tokenized_sentence.index("[MASK]")
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_sentence)
    token_tensor = torch.tensor([token_ids]).to('cuda')

    # Forward pass without training and backpropagation
    with torch.no_grad():
        outputs = modek(token_tensor)
        prediction_logits = outputs[0]
    
    # Calculating 3 nearest prediction of masked token 
    prediction_probs = torch.nn.functional.softmax(prediction_logits[0, mask_index], dim=-1)
    top_k_probs, top_k_token_ids = torch.topk(prediction_probs, 3, sorted=True)

    # Print the predicted tokens for mask and their weight
    predicted_mask = []
    for i in range(3):
        predicted_mask.append(tokenizer.convert_ids_to_tokens([top_k_token_ids[i]])[0])
        token_prob = top_k_probs[i].item()
        print(f"[MASK]: '{predicted_mask[i]}'  : {token_prob}")       
    return predicted_mask[0]   

In [32]:
import pandas as pd
df = pd.read_csv("/content/Test_file.txt", engine='python',encoding='utf-8', error_bad_lines=False)
df.columns = ["cat", "first", "second", "third","fourth"]
df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,cat,first,second,third,fourth
0,city,اهواز,خوزستان,قشم,هرمزگان
1,city,شیراز,فارس,دامغان,سمنان
2,city,رشت,گیلان,اسلام‌شهر,تهران
3,city,بندرعباس,هرمزگان,ملارد,تهران
4,city,اراک,مرکزی,مریوان,کردستان


In [33]:
Accuracy_city = 0
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "شهر "+df["first"][i[0]] + " در استان " + df["second"][i[0]] + " است در نتیجه شهر " + df["third"][i[0]] + " در استان " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_city = Accuracy_city + 1   
print(Accuracy_city)        



اهواز خوزستان قشم هرمزگان
شهر اهواز در استان خوزستان است در نتیجه شهر قشم در استان [MASK] است.
[MASK]: '##றறில'  : 0.06756976246833801
[MASK]: 'тех'  : 0.05561196431517601
[MASK]: '##లనూ'  : 0.049346208572387695
شیراز فارس دامغان سمنان
شهر شیراز در استان فارس است در نتیجه شهر دامغان در استان [MASK] است.
[MASK]: '缽'  : 0.050481248646974564
[MASK]: '##లనూ'  : 0.04616234451532364
[MASK]: 'тех'  : 0.03791768476366997
رشت گیلان اسلام‌شهر تهران
شهر رشت در استان گیلان است در نتیجه شهر اسلام‌شهر در استان [MASK] است.
[MASK]: '##లనూ'  : 0.12175749242305756
[MASK]: 'spolecnost'  : 0.03063785284757614
[MASK]: '##றறில'  : 0.027496928349137306
بندرعباس هرمزگان ملارد تهران
شهر بندرعباس در استان هرمزگان است در نتیجه شهر ملارد در استان [MASK] است.
[MASK]: '##లనూ'  : 0.09583596885204315
[MASK]: 'формула'  : 0.05243230238556862
[MASK]: 'tartozo'  : 0.02134685218334198
اراک مرکزی مریوان کردستان
شهر اراک در استان مرکزی است در نتیجه شهر مریوان در استان [MASK] است.
[MASK]: '##లనూ'  : 0.08481938391923904
[MAS

In [35]:
Accuracy = []
Accuracy_city = 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "شهر "+df["first"][i[0]] + " در استان " + df["second"][i[0]] + "  در نتیجه شهر " + df["third"][i[0]] + " در استان " + "[MASK]" + " می باشد."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_city = Accuracy_city + 1   
num = y["cat"].count()      
print(Accuracy_city/num)      



اهواز خوزستان قشم هرمزگان
شهر اهواز در استان خوزستان  در نتیجه شهر قشم در استان [MASK] می باشد.
[MASK]: '##றறில'  : 0.08135715126991272
[MASK]: 'filles'  : 0.061181630939245224
[MASK]: '##లనూ'  : 0.028923483565449715
شیراز فارس دامغان سمنان
شهر شیراز در استان فارس  در نتیجه شهر دامغان در استان [MASK] می باشد.
[MASK]: 'tartozo'  : 0.037546444684267044
[MASK]: '##cripcio'  : 0.03667674958705902
[MASK]: '##றறில'  : 0.025075044482946396
رشت گیلان اسلام‌شهر تهران
شهر رشت در استان گیلان  در نتیجه شهر اسلام‌شهر در استان [MASK] می باشد.
[MASK]: '##లనూ'  : 0.04408150538802147
[MASK]: '缽'  : 0.038161661475896835
[MASK]: 'tartozo'  : 0.03748098388314247
بندرعباس هرمزگان ملارد تهران
شهر بندرعباس در استان هرمزگان  در نتیجه شهر ملارد در استان [MASK] می باشد.
[MASK]: '##లనూ'  : 0.14269442856311798
[MASK]: '##றறில'  : 0.026917658746242523
[MASK]: 'тех'  : 0.026867186650633812
اراک مرکزی مریوان کردستان
شهر اراک در استان مرکزی  در نتیجه شهر مریوان در استان [MASK] می باشد.
[MASK]: '##లనూ'  : 0.0539261847

In [36]:
Accuracy_currency = 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " واحد پول "+df["second"][i[0]] + " برای کشور " + df["first"][i[0]] + " است پس ارز و پول " + "[MASK]" + " برای کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_currency = Accuracy_currency + 1   
num = y["cat"].count()      
print(Accuracy_currency/num)     

سوئد کرون امارات درهم
 واحد پول کرون برای کشور سوئد است پس ارز و پول [MASK] برای کشور امارات است.
[MASK]: '##లనూ'  : 0.06263787299394608
[MASK]: 'wade'  : 0.05196736007928848
[MASK]: 'тех'  : 0.02274390310049057
سوئیس فرانک تاجیکستان سامانی
 واحد پول فرانک برای کشور سوئیس است پس ارز و پول [MASK] برای کشور تاجیکستان است.
[MASK]: '##றறில'  : 0.037804584950208664
[MASK]: 'yleensa'  : 0.023692885413765907
[MASK]: '##юцца'  : 0.018227607011795044
رومانی لئو بریتانیا پوند
 واحد پول لئو برای کشور رومانی است پس ارز و پول [MASK] برای کشور بریتانیا است.
[MASK]: '##లనూ'  : 0.12056074291467667
[MASK]: 'тех'  : 0.056544605642557144
[MASK]: 'filles'  : 0.04758119583129883
ترکیه لیره افغانستان افغانی
 واحد پول لیره برای کشور ترکیه است پس ارز و پول [MASK] برای کشور افغانستان است.
[MASK]: '##లనూ'  : 0.07894118875265121
[MASK]: 'tartozo'  : 0.03106311336159706
[MASK]: '##றறில'  : 0.029091788455843925
چین یوان عراق دینار
 واحد پول یوان برای کشور چین است پس ارز و پول [MASK] برای کشور عراق است.
[MASK]: '##

In [37]:
Accuracy_capital = 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " شهر "+df["second"][i[0]] + " پایتخت کشور " + df["first"][i[0]] + " است پس شهر " + "[MASK]" + " پایتخت کشور " + df["third"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_capital = Accuracy_capital + 1   
num = y["cat"].count()      
print(Accuracy_capital/num)     

روسیه مسکو هلند آمستردام
 شهر مسکو پایتخت کشور روسیه است پس شهر [MASK] پایتخت کشور هلند است.
[MASK]: 'tartozo'  : 0.05216744542121887
[MASK]: 'формула'  : 0.029786543920636177
[MASK]: '缽'  : 0.02877192758023739
کره‌جنوبی سئول آمریکا واشنگتن
 شهر سئول پایتخت کشور کره‌جنوبی است پس شهر [MASK] پایتخت کشور آمریکا است.
[MASK]: '##లనూ'  : 0.12766945362091064
[MASK]: '##றறில'  : 0.040206167846918106
[MASK]: 'tartozo'  : 0.024242553859949112
ایران تهران اسپانیا مادرید
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور اسپانیا است.
[MASK]: '##లనూ'  : 0.07865338772535324
[MASK]: 'тех'  : 0.03633371740579605
[MASK]: '##றறில'  : 0.031218620017170906
ایران تهران افغانستان کابل
 شهر تهران پایتخت کشور ایران است پس شهر [MASK] پایتخت کشور افغانستان است.
[MASK]: '##లనూ'  : 0.13302014768123627
[MASK]: 'temporadas'  : 0.02736687660217285
[MASK]: '##றறில'  : 0.025739319622516632
بریتانیا لندن فنلاند هلسینکی
 شهر لندن پایتخت کشور بریتانیا است پس شهر [MASK] پایتخت کشور فنلاند است.
[MASK]: '##లనూ'  : 0

In [38]:
Accuracy_family = 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت خانوادگی "+df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت خانوادگی " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy_family = Accuracy_family + 1   
num = y["cat"].count()      
print(Accuracy_family/num)     

برادر خواهر پدر مادر
 نسبت خانوادگی برادر به خواهر مانند نسبت خانوادگی پدر به [MASK] است.
[MASK]: '##లనూ'  : 0.0762408971786499
[MASK]: 'wade'  : 0.02131744846701622
[MASK]: 'тех'  : 0.0204667616635561
برادرها خواهرها شوهر عیال
 نسبت خانوادگی برادرها به خواهرها مانند نسبت خانوادگی شوهر به [MASK] است.
[MASK]: '##లనూ'  : 0.07077741622924805
[MASK]: 'filles'  : 0.036278799176216125
[MASK]: 'tartozo'  : 0.029943563044071198
پدربزرگ مادربزرگ برادر خواهر
 نسبت خانوادگی پدربزرگ به مادربزرگ مانند نسبت خانوادگی برادر به [MASK] است.
[MASK]: '##юцца'  : 0.026048999279737473
[MASK]: 'এহানর'  : 0.018475504592061043
[MASK]: 'spolecnost'  : 0.017779076471924782
بابابزرگ مامان‌بزرگ داماد عروس
 نسبت خانوادگی بابابزرگ به مامان‌بزرگ مانند نسبت خانوادگی داماد به [MASK] است.
[MASK]: 'tartozo'  : 0.05933814495801926
[MASK]: '##లనూ'  : 0.03789855167269707
[MASK]: '##றறில'  : 0.03361451253294945
داماد عروس برادرها خواهرها
 نسبت خانوادگی داماد به عروس مانند نسبت خانوادگی برادرها به [MASK] است.
[MASK]: '##లనూ' 

In [39]:
Accuracy = 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " فعل "+df["first"][i[0]] + " سوم شخص مفرد و فعل " + df["second"][i[0]] + " سوم شخص جمع است در نتیجه فعل " + df["third"][i[0]] + " سوم شخص مفرد و فعل " + "[MASK]" + " سوم شخص جمع است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)     

برید بریدند رقصید رقصیدند
 فعل برید سوم شخص مفرد و فعل بریدند سوم شخص جمع است در نتیجه فعل رقصید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: '##లనూ'  : 0.05121997743844986
[MASK]: 'tartozo'  : 0.034221209585666656
[MASK]: 'тех'  : 0.026050115004181862
پرید پریدند آمد آمدند
 فعل پرید سوم شخص مفرد و فعل پریدند سوم شخص جمع است در نتیجه فعل آمد سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: '##లనూ'  : 0.03467226400971413
[MASK]: 'tartozo'  : 0.03422664478421211
[MASK]: 'тех'  : 0.024385331198573112
تازید تازیدند گفت گفتند
 فعل تازید سوم شخص مفرد و فعل تازیدند سوم شخص جمع است در نتیجه فعل گفت سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: 'spolecnost'  : 0.029420550912618637
[MASK]: 'temporadas'  : 0.023057641461491585
[MASK]: 'filles'  : 0.02039768546819687
شد شدند بوسید بوسیدند
 فعل شد سوم شخص مفرد و فعل شدند سوم شخص جمع است در نتیجه فعل بوسید سوم شخص مفرد و فعل [MASK] سوم شخص جمع است.
[MASK]: '##లనూ'  : 0.14046722650527954
[MASK]: 'spolecnost'  : 0.03299583122134209
[MASK]

In [40]:
Accuracy = 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  مصدر "+df["third"][i[0]] + " فعل " + "[MASK]" + " است همانطور که مصدر " + df["first"][i[0]] + " فعل " + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)      

بریدن برید دمیدن دمید
  مصدر دمیدن فعل [MASK] است همانطور که مصدر بریدن فعل برید است.
[MASK]: 'формула'  : 0.048918452113866806
[MASK]: 'tartozo'  : 0.02834813855588436
[MASK]: 'possivel'  : 0.027378864586353302
تازیدن تاخت دادن داد
  مصدر دادن فعل [MASK] است همانطور که مصدر تازیدن فعل تاخت است.
[MASK]: '##లనూ'  : 0.12596097588539124
[MASK]: 'тех'  : 0.08784624934196472
[MASK]: 'tartozo'  : 0.02608027309179306
شدن شد نگریستن نگریست
  مصدر نگریستن فعل [MASK] است همانطور که مصدر شدن فعل شد است.
[MASK]: 'wade'  : 0.04809519648551941
[MASK]: 'тех'  : 0.023328445851802826
[MASK]: '##లనూ'  : 0.02282913401722908
گفتن گفت رقصیدن رقصید
  مصدر رقصیدن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: '##లనూ'  : 0.13879628479480743
[MASK]: 'тех'  : 0.046947527676820755
[MASK]: 'filles'  : 0.03580296412110329
گفتن گفت خواستن خواست
  مصدر خواستن فعل [MASK] است همانطور که مصدر گفتن فعل گفت است.
[MASK]: '##లనూ'  : 0.18718960881233215
[MASK]: '##றறில'  : 0.022610653191804886
[MASK]: 'тех'  : 0.0

In [41]:
Accuracy = 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " من "+df["third"][i[0]] + "  پس ما " + "[MASK]" + " من " + df["first"][i[0]] + " و ما " + df["second"][i[0]] + " "
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

دادم دادیم یافتم یافتیم
 من یافتم  پس ما [MASK] من دادم و ما دادیم 
[MASK]: '##లనూ'  : 0.05801210552453995
[MASK]: 'формула'  : 0.036861445754766464
[MASK]: '##றறில'  : 0.02461189590394497
زدم زدیم نامیدم نامیدیم
 من نامیدم  پس ما [MASK] من زدم و ما زدیم 
[MASK]: 'формула'  : 0.05841285362839699
[MASK]: '##లనూ'  : 0.05517437309026718
[MASK]: 'тех'  : 0.03250247240066528
نوشتم نوشتیم گفتم گفتیم
 من گفتم  پس ما [MASK] من نوشتم و ما نوشتیم 
[MASK]: '##లనూ'  : 0.04852614924311638
[MASK]: 'tartozo'  : 0.04243357479572296
[MASK]: 'filles'  : 0.020071938633918762
دانستم دانستیم بلعیدم بلعیدیم
 من بلعیدم  پس ما [MASK] من دانستم و ما دانستیم 
[MASK]: '##లనూ'  : 0.08425196260213852
[MASK]: '##றறில'  : 0.03645426779985428
[MASK]: 'territoires'  : 0.033378127962350845
گردیدم گردیدیم یافتم یافتیم
 من یافتم  پس ما [MASK] من گردیدم و ما گردیدیم 
[MASK]: '##లనూ'  : 0.05802825838327408
[MASK]: 'tartozo'  : 0.03278330713510513
[MASK]: 'формула'  : 0.027725519612431526
گردیدم گردیدیم بوسیدم بوسیدیم
 من ب

In [42]:
Accuracy = 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "  "+ df["first"][i[0]] + " صفت و " + df["second"][i[0]] + " قید است. در نتیجه " + df["third"][i[0]] + " صفت و " + "[MASK]" + " قید است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

مجدد مجددا مخصوص مخصوصا
  مجدد صفت و مجددا قید است. در نتیجه مخصوص صفت و [MASK] قید است.
[MASK]: '##றறில'  : 0.0439479798078537
[MASK]: 'tartozo'  : 0.03895781189203262
[MASK]: '##లనూ'  : 0.034232206642627716
عمده عمدتا صمیمی صمیمانه
  عمده صفت و عمدتا قید است. در نتیجه صمیمی صفت و [MASK] قید است.
[MASK]: '##లనూ'  : 0.05124807357788086
[MASK]: 'тех'  : 0.040939345955848694
[MASK]: 'wade'  : 0.030543310567736626
دقیق دقیقا عادل عادلانه
  دقیق صفت و دقیقا قید است. در نتیجه عادل صفت و [MASK] قید است.
[MASK]: '##లనూ'  : 0.07603684812784195
[MASK]: 'тех'  : 0.048374246805906296
[MASK]: 'descenso'  : 0.029208382591605186
غالب غالبا جدا جداگانه
  غالب صفت و غالبا قید است. در نتیجه جدا صفت و [MASK] قید است.
[MASK]: '##లనూ'  : 0.03699871897697449
[MASK]: '##றறில'  : 0.03603292629122734
[MASK]: 'tartozo'  : 0.022110480815172195
متناظر متناظرا قبل قبلا
  متناظر صفت و متناظرا قید است. در نتیجه قبل صفت و [MASK] قید است.
[MASK]: '##లనూ'  : 0.06484251469373703
[MASK]: 'тех'  : 0.03706119582056999
[MA

In [43]:
Accuracy = 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتفاق اتفاقا شب شبانه
 نسبت اتفاق به اتفاقا مانند نسبت شب به [MASK] است.
[MASK]: '##లనూ'  : 0.11713746190071106
[MASK]: 'тех'  : 0.052733078598976135
[MASK]: 'tartozo'  : 0.033303722739219666
حتم حتما اساس اساسا
 نسبت حتم به حتما مانند نسبت اساس به [MASK] است.
[MASK]: '缽'  : 0.07899008691310883
[MASK]: '##లనూ'  : 0.051458392292261124
[MASK]: 'тех'  : 0.03380599245429039
اصل اصلا انسان انسانوار
 نسبت اصل به اصلا مانند نسبت انسان به [MASK] است.
[MASK]: '##లనూ'  : 0.3175630271434784
[MASK]: 'тех'  : 0.025728750973939896
[MASK]: '##றறில'  : 0.02437385357916355
عمل عملا اجبار اجبارا
 نسبت عمل به عملا مانند نسبت اجبار به [MASK] است.
[MASK]: '##లనూ'  : 0.06622236967086792
[MASK]: '##றறில'  : 0.03848126903176308
[MASK]: 'tartozo'  : 0.03686181828379631
اجبار اجبارا کودک کودکانه
 نسبت اجبار به اجبارا مانند نسبت کودک به [MASK] است.
[MASK]: '##లనూ'  : 0.05443916842341423
[MASK]: 'тех'  : 0.03963802009820938
[MASK]: 'descenso'  : 0.024559801444411278
دائم دائما اتفاق اتفاقا
 نسبت دائم به دائما مان

In [44]:
Accuracy = 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

خشنود ناخشنود موفق ناموفق
خشنود به ناخشنود مانند موفق به [MASK] است.
[MASK]: '##లనూ'  : 0.139804407954216
[MASK]: 'tartozo'  : 0.04727400094270706
[MASK]: 'spolecnost'  : 0.020078444853425026
درست نادرست پخته ناپخته
درست به نادرست مانند پخته به [MASK] است.
[MASK]: '##లనూ'  : 0.14172466099262238
[MASK]: 'тех'  : 0.03512692078948021
[MASK]: 'tartozo'  : 0.021369855850934982
مناسب نامناسب همزمان ناهمزمان
مناسب به نامناسب مانند همزمان به [MASK] است.
[MASK]: '##లనూ'  : 0.05657285079360008
[MASK]: 'poesia'  : 0.0399363711476326
[MASK]: 'wade'  : 0.036613579839468
مربوط نامربوط منسجم نامنسجم
مربوط به نامربوط مانند منسجم به [MASK] است.
[MASK]: 'wade'  : 0.06504137814044952
[MASK]: '##లనూ'  : 0.023089176043868065
[MASK]: '##றறில'  : 0.022178569808602333
بینا نابینا درست نادرست
بینا به نابینا مانند درست به [MASK] است.
[MASK]: '##లనూ'  : 0.148642435669899
[MASK]: 'тех'  : 0.02732512727379799
[MASK]: '##றறில'  : 0.026529617607593536
کارآمد ناکارآمد هنجار ناهنجار
کارآمد به ناکارآمد مانند هنجار به [

In [45]:
Accuracy = 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["first"][i[0]] + " به " + df["second"][i[0]] + " مانند نسبت " + df["third"][i[0]] + " به " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

سریع سریعتر بلند بلندتر
 نسبت سریع به سریعتر مانند نسبت بلند به [MASK] است.
[MASK]: '##లనూ'  : 0.10745670646429062
[MASK]: 'tartozo'  : 0.02195350080728531
[MASK]: 'тех'  : 0.020336339250206947
بزرگ بزرگتر شدید شدیدتر
 نسبت بزرگ به بزرگتر مانند نسبت شدید به [MASK] است.
[MASK]: 'filles'  : 0.03874426335096359
[MASK]: '##లనూ'  : 0.028815777972340584
[MASK]: 'tartozo'  : 0.024571191519498825
مهم مهمتر بارز بارزتر
 نسبت مهم به مهمتر مانند نسبت بارز به [MASK] است.
[MASK]: '##లనూ'  : 0.08967939019203186
[MASK]: 'tartozo'  : 0.03704351559281349
[MASK]: 'тех'  : 0.03434326499700546
به بهتر سرد سردتر
 نسبت به به بهتر مانند نسبت سرد به [MASK] است.
[MASK]: '##లనూ'  : 0.14322279393672943
[MASK]: 'тех'  : 0.07358766347169876
[MASK]: 'filles'  : 0.06054610759019852
واضح واضحتر بیش بیشتر
 نسبت واضح به واضحتر مانند نسبت بیش به [MASK] است.
[MASK]: '##లనూ'  : 0.12472208589315414
[MASK]: 'тех'  : 0.04327736049890518
[MASK]: 'filles'  : 0.021168677136301994
کم کمتر بد بدتر
 نسبت کم به کمتر مانند نسبت بد ب

In [46]:
Accuracy = 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " مردم " + df["second"][i[0]] + " اهل کشور " + df["first"][i[0]] + " هستند. و مردم " + "[MASK]" + " اهل کشور " + df["third"][i[0]] + " هستند."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

اتریش اتریشی اسپانیا اسپانیایی
 مردم اتریشی اهل کشور اتریش هستند. و مردم [MASK] اهل کشور اسپانیا هستند.
[MASK]: '##లనూ'  : 0.15946422517299652
[MASK]: '##nden'  : 0.028453975915908813
[MASK]: 'тех'  : 0.017620932310819626
سوئد سوئدی فنلاند فنلاندی
 مردم سوئدی اهل کشور سوئد هستند. و مردم [MASK] اهل کشور فنلاند هستند.
[MASK]: '缽'  : 0.037463825196027756
[MASK]: 'тех'  : 0.0349799208343029
[MASK]: '##цкого'  : 0.030675161629915237
بحرین بحرینی عراق عراقی
 مردم بحرینی اهل کشور بحرین هستند. و مردم [MASK] اهل کشور عراق هستند.
[MASK]: '##లనూ'  : 0.13096921145915985
[MASK]: 'тех'  : 0.03513892740011215
[MASK]: 'wade'  : 0.03097393363714218
کویت کویتی سوریه سوری
 مردم کویتی اهل کشور کویت هستند. و مردم [MASK] اهل کشور سوریه هستند.
[MASK]: '缽'  : 0.05384884774684906
[MASK]: '##றறில'  : 0.029751377180218697
[MASK]: 'tartozo'  : 0.02528836950659752
مصر مصری کرواسی کروات
 مردم مصری اهل کشور مصر هستند. و مردم [MASK] اهل کشور کرواسی هستند.
[MASK]: '##றறில'  : 0.04581514000892639
[MASK]: 'tartozo'  : 0

In [47]:
Accuracy = 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #Capital
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " " + df["first"][i[0]] + " جمع " + df["second"][i[0]] + " است و " + df["third"][i[0]] + " جمع " + "[MASK]" + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    if (predicted == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/num)   

تالیف تالیفات مخلوق مخلوقات
 تالیف جمع تالیفات است و مخلوق جمع [MASK] است.
[MASK]: '##లనూ'  : 0.08820362389087677
[MASK]: 'тех'  : 0.017075391486287117
[MASK]: 'filles'  : 0.016039835289120674
اثر اثرات ارزش ارزشها
 اثر جمع اثرات است و ارزش جمع [MASK] است.
[MASK]: '##లనూ'  : 0.17308871448040009
[MASK]: '##юцца'  : 0.04791131615638733
[MASK]: 'тех'  : 0.038855161517858505
انقلابی انقلابیون مورد موارد
 انقلابی جمع انقلابیون است و مورد جمع [MASK] است.
[MASK]: '##లనూ'  : 0.19403865933418274
[MASK]: 'tartozo'  : 0.03608347848057747
[MASK]: 'тех'  : 0.03182230517268181
حواری حواریون مورد موارد
 حواری جمع حواریون است و مورد جمع [MASK] است.
[MASK]: 'тех'  : 0.10724171251058578
[MASK]: '##లనూ'  : 0.09049779921770096
[MASK]: 'territoires'  : 0.05313307046890259
مسئول مسئولین آزاده آزادگان
 مسئول جمع مسئولین است و آزاده جمع [MASK] است.
[MASK]: '##లనూ'  : 0.05545886605978012
[MASK]: 'spolecnost'  : 0.03800949081778526
[MASK]: 'yleensa'  : 0.028324266895651817
مسافر مسافرین طلا طلاها
 مسافر جمع مسا

**تست روش دوم**

In [48]:
Accuracy= 0
y= df[df["cat"]=="city"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "city"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " "+df["first"][i[0]] + " در " + "[MASK]" + df["second"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = " "+df["third"][i[0]] + " در " + predicted + "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)      


اهواز خوزستان قشم هرمزگان
 اهواز در [MASK]خوزستان است.
[MASK]: '##లనూ'  : 0.10204895585775375
[MASK]: 'tartozo'  : 0.04263130947947502
[MASK]: 'spolecnost'  : 0.02915181964635849
 قشم در ##లనూ[MASK] است.
[MASK]: '##లనూ'  : 0.09092707186937332
[MASK]: 'формула'  : 0.03110559470951557
[MASK]: '##றறில'  : 0.031037088483572006
شیراز فارس دامغان سمنان
 شیراز در [MASK]فارس است.
[MASK]: '##లనూ'  : 0.04375724866986275
[MASK]: 'tartozo'  : 0.0346108078956604
[MASK]: 'temporadas'  : 0.027548808604478836
 دامغان در ##లనూ[MASK] است.
[MASK]: '##లనూ'  : 0.043714966624975204
[MASK]: '##றறில'  : 0.03907076269388199
[MASK]: 'формула'  : 0.037797171622514725
رشت گیلان اسلام‌شهر تهران
 رشت در [MASK]گیلان است.
[MASK]: 'тех'  : 0.06116669625043869
[MASK]: '##లనూ'  : 0.057662371546030045
[MASK]: 'tartozo'  : 0.03133751079440117
 اسلام‌شهر در тех[MASK] است.
[MASK]: 'тех'  : 0.07708713412284851
[MASK]: '##లనూ'  : 0.05435086414217949
[MASK]: '##றறில'  : 0.028951210901141167
بندرعباس هرمزگان ملارد تهران
 بندرعب

In [49]:
Accuracy= 0
y= df[df["cat"]=="capital"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "capital"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = " "+"[MASK]" + " " + predicted + " " + df["third"][i[0]] + " است."
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

روسیه مسکو هلند آمستردام
مسکو [MASK] روسیه است.
[MASK]: '##లనూ'  : 0.12119174003601074
[MASK]: '##றறில'  : 0.028195220977067947
[MASK]: 'tartozo'  : 0.023224007338285446
 [MASK] ##లనూ هلند است.
[MASK]: '##లనూ'  : 0.11600204557180405
[MASK]: 'tartozo'  : 0.025735871866345406
[MASK]: 'われている'  : 0.01957351714372635
کره‌جنوبی سئول آمریکا واشنگتن
سئول [MASK] کره‌جنوبی است.
[MASK]: '##లనూ'  : 0.05959252640604973
[MASK]: 'тех'  : 0.026986267417669296
[MASK]: 'формула'  : 0.02628757432103157
 [MASK] ##లనూ آمریکا است.
[MASK]: '##లనూ'  : 0.09423937648534775
[MASK]: 'тех'  : 0.03810535743832588
[MASK]: 'tartozo'  : 0.030682940036058426
ایران تهران اسپانیا مادرید
تهران [MASK] ایران است.
[MASK]: 'тех'  : 0.054355084896087646
[MASK]: '##тің'  : 0.02358359843492508
[MASK]: '##లనూ'  : 0.022905055433511734
 [MASK] тех اسپانیا است.
[MASK]: '##లనూ'  : 0.10597097128629684
[MASK]: '##юцца'  : 0.06871505081653595
[MASK]: '缽'  : 0.0218056607991457
ایران تهران افغانستان کابل
تهران [MASK] ایران است.
[MASK]: '#

In [50]:
Accuracy= 0
y= df[df["cat"]=="currency"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "currency"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است."
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)   

سوئد کرون امارات درهم
کرون [MASK] سوئد است.
[MASK]: 'тех'  : 0.03462876379489899
[MASK]: '##లనూ'  : 0.031918734312057495
[MASK]: 'territoires'  : 0.02505837008357048
техامارات [MASK] است.
[MASK]: 'тех'  : 0.030139649286866188
[MASK]: '##юцца'  : 0.024808447808027267
[MASK]: '##లనూ'  : 0.01752937212586403
سوئیس فرانک تاجیکستان سامانی
فرانک [MASK] سوئیس است.
[MASK]: '##లనూ'  : 0.08779002726078033
[MASK]: 'wade'  : 0.03512781485915184
[MASK]: '缽'  : 0.027561519294977188
##లనూتاجیکستان [MASK] است.
[MASK]: '##లనూ'  : 0.08082786202430725
[MASK]: 'тех'  : 0.06446556746959686
[MASK]: '缽'  : 0.04557780921459198
رومانی لئو بریتانیا پوند
لئو [MASK] رومانی است.
[MASK]: '##లనూ'  : 0.03154245764017105
[MASK]: 'тех'  : 0.02863670140504837
[MASK]: 'filles'  : 0.025323139503598213
##లనూبریتانیا [MASK] است.
[MASK]: 'тех'  : 0.03178010135889053
[MASK]: '##లనూ'  : 0.02736072614789009
[MASK]: '##றறில'  : 0.01590735837817192
ترکیه لیره افغانستان افغانی
لیره [MASK] ترکیه است.
[MASK]: '##లనూ'  : 0.09203965961

In [51]:
Accuracy= 0
y= df[df["cat"]=="family"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "family"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برادر خواهر پدر مادر
خواهر [MASK] برادر است.
[MASK]: 'тех'  : 0.06416062265634537
[MASK]: 'tartozo'  : 0.039004553109407425
[MASK]: '##లనూ'  : 0.03533749654889107
техپدر [MASK] است. 
[MASK]: '##లనూ'  : 0.05840571969747543
[MASK]: '##றறில'  : 0.036580123007297516
[MASK]: 'tartozo'  : 0.0287934523075819
برادرها خواهرها شوهر عیال
خواهرها [MASK] برادرها است.
[MASK]: '##లనూ'  : 0.0454254113137722
[MASK]: 'тех'  : 0.03305850178003311
[MASK]: 'формула'  : 0.028541510924696922
##లనూشوهر [MASK] است. 
[MASK]: '##లనూ'  : 0.0934617668390274
[MASK]: '缽'  : 0.0306602343916893
[MASK]: 'spolecnost'  : 0.025515425950288773
پدربزرگ مادربزرگ برادر خواهر
مادربزرگ [MASK] پدربزرگ است.
[MASK]: '##లనూ'  : 0.11960804462432861
[MASK]: 'spolecnost'  : 0.04498547688126564
[MASK]: 'tartozo'  : 0.041605472564697266
##లనూبرادر [MASK] است. 
[MASK]: '##లనూ'  : 0.12652838230133057
[MASK]: 'тех'  : 0.04273594543337822
[MASK]: '##றறில'  : 0.035517096519470215
بابابزرگ مامان‌بزرگ داماد عروس
مامان‌بزرگ [MASK] بابابزرگ است.

In [52]:
Accuracy= 0
y= df[df["cat"]=="gram_thirdperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_thirdperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = " نسبت " + df["second"][i[0]] + " به " +df["first"][i[0]]  + " " + "[MASK]"+ " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

برید بریدند رقصید رقصیدند
 نسبت بریدند به برید [MASK] است.
[MASK]: '##లనూ'  : 0.04631438106298447
[MASK]: 'filles'  : 0.03817463666200638
[MASK]: 'тех'  : 0.03502419963479042
##లనూرقصید [MASK] است. 
[MASK]: '##றறில'  : 0.04165203496813774
[MASK]: 'wade'  : 0.027751052752137184
[MASK]: 'filles'  : 0.026753775775432587
پرید پریدند آمد آمدند
 نسبت پریدند به پرید [MASK] است.
[MASK]: '缽'  : 0.13553430140018463
[MASK]: '##లనూ'  : 0.05543813109397888
[MASK]: 'wade'  : 0.04322279989719391
缽آمد [MASK] است. 
[MASK]: '##లనూ'  : 0.20301632583141327
[MASK]: 'тех'  : 0.0287494957447052
[MASK]: '##றறில'  : 0.026053931564092636
تازید تازیدند گفت گفتند
 نسبت تازیدند به تازید [MASK] است.
[MASK]: '##లనూ'  : 0.08551724255084991
[MASK]: 'тех'  : 0.04904036968946457
[MASK]: '##றறில'  : 0.02654501423239708
##లనూگفت [MASK] است. 
[MASK]: '##றறில'  : 0.0616409033536911
[MASK]: '##లనూ'  : 0.06006364896893501
[MASK]: 'territoires'  : 0.027521757408976555
شد شدند بوسید بوسیدند
 نسبت شدند به شد [MASK] است.
[MASK]: 

In [53]:
Accuracy= 0
y= df[df["cat"]=="gram_past"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_past"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

بریدن برید دمیدن دمید
برید [MASK] بریدن است.
[MASK]: '##లనూ'  : 0.07508967071771622
[MASK]: '##றறில'  : 0.028131410479545593
[MASK]: '缽'  : 0.020720480009913445
##లనూدمیدن [MASK] است. 
[MASK]: '##లనూ'  : 0.18028490245342255
[MASK]: 'тех'  : 0.052037548273801804
[MASK]: 'filles'  : 0.025788672268390656
تازیدن تاخت دادن داد
تاخت [MASK] تازیدن است.
[MASK]: '##లనూ'  : 0.057947203516960144
[MASK]: '##றறில'  : 0.05162806063890457
[MASK]: '缽'  : 0.03011995181441307
##లనూدادن [MASK] است. 
[MASK]: '##లనూ'  : 0.19195061922073364
[MASK]: 'тех'  : 0.028373045846819878
[MASK]: 'tartozo'  : 0.02800576761364937
شدن شد نگریستن نگریست
شد [MASK] شدن است.
[MASK]: '##లనూ'  : 0.06719982624053955
[MASK]: '缽'  : 0.04153134301304817
[MASK]: 'тех'  : 0.025441747158765793
##లనూنگریستن [MASK] است. 
[MASK]: '##றறில'  : 0.03892103210091591
[MASK]: '##లనూ'  : 0.03249320387840271
[MASK]: 'тех'  : 0.026303911581635475
گفتن گفت رقصیدن رقصید
گفت [MASK] گفتن است.
[MASK]: '##లనూ'  : 0.10381404310464859
[MASK]: '##றறில'  

In [54]:
Accuracy= 0
y= df[df["cat"]=="gram_firstperson"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_firstperson"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

دادم دادیم یافتم یافتیم
دادیم [MASK] دادم است.
[MASK]: '##లనూ'  : 0.06835994869470596
[MASK]: 'формула'  : 0.03433186933398247
[MASK]: '##றறில'  : 0.03052922524511814
##లనూیافتم [MASK] است. 
[MASK]: 'wade'  : 0.04428481683135033
[MASK]: '缽'  : 0.04169769957661629
[MASK]: '##లనూ'  : 0.03357144445180893
زدم زدیم نامیدم نامیدیم
زدیم [MASK] زدم است.
[MASK]: '##లనూ'  : 0.11722397804260254
[MASK]: 'tartozo'  : 0.036535222083330154
[MASK]: '##юцца'  : 0.01986067369580269
##లనూنامیدم [MASK] است. 
[MASK]: '##లనూ'  : 0.13642024993896484
[MASK]: 'filles'  : 0.04840271547436714
[MASK]: 'tartozo'  : 0.024134689942002296
نوشتم نوشتیم گفتم گفتیم
نوشتیم [MASK] نوشتم است.
[MASK]: '##లనూ'  : 0.1363920420408249
[MASK]: 'tartozo'  : 0.02860770747065544
[MASK]: 'формула'  : 0.024298734962940216
##లనూگفتم [MASK] است. 
[MASK]: '##లనూ'  : 0.08943711966276169
[MASK]: 'tartozo'  : 0.045192621648311615
[MASK]: 'тех'  : 0.042634934186935425
دانستم دانستیم بلعیدم بلعیدیم
دانستیم [MASK] دانستم است.
[MASK]: '##లనూ' 

In [55]:
Accuracy= 0
y= df[df["cat"]=="gram_adj2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_adj2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

مجدد مجددا مخصوص مخصوصا
مجددا [MASK] مجدد است.
[MASK]: '##లనూ'  : 0.053802985697984695
[MASK]: 'filles'  : 0.032192617654800415
[MASK]: 'tartozo'  : 0.02713756449520588
##లనూمخصوص [MASK] است. 
[MASK]: '##లనూ'  : 0.21110126376152039
[MASK]: '##றறில'  : 0.02160186693072319
[MASK]: 'тех'  : 0.01545630767941475
عمده عمدتا صمیمی صمیمانه
عمدتا [MASK] عمده است.
[MASK]: '##లనూ'  : 0.05892166867852211
[MASK]: '缽'  : 0.024442775174975395
[MASK]: 'tartozo'  : 0.02064681425690651
##లనూصمیمی [MASK] است. 
[MASK]: '##లనూ'  : 0.10058924555778503
[MASK]: 'тех'  : 0.04962452873587608
[MASK]: 'tartozo'  : 0.04862721636891365
دقیق دقیقا عادل عادلانه
دقیقا [MASK] دقیق است.
[MASK]: '##లనూ'  : 0.06503548473119736
[MASK]: 'тех'  : 0.034209638833999634
[MASK]: 'tartozo'  : 0.0339537151157856
##లనూعادل [MASK] است. 
[MASK]: '##లనూ'  : 0.11922192573547363
[MASK]: 'тех'  : 0.04091833531856537
[MASK]: '##றறில'  : 0.039577797055244446
غالب غالبا جدا جداگانه
غالبا [MASK] غالب است.
[MASK]: '缽'  : 0.03476919233798981
[

In [56]:
Accuracy= 0
y= df[df["cat"]=="gram_noun2adv"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_noun2adv"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

اتفاق اتفاقا شب شبانه
اتفاقا [MASK] اتفاق است.
[MASK]: '##లనూ'  : 0.07420637458562851
[MASK]: '##றறில'  : 0.056590255349874496
[MASK]: 'territoires'  : 0.02000676468014717
##లనూشب [MASK] است. 
[MASK]: 'тех'  : 0.1049574688076973
[MASK]: '##లనూ'  : 0.057775598019361496
[MASK]: '##юцца'  : 0.02775680087506771
حتم حتما اساس اساسا
حتما [MASK] حتم است.
[MASK]: '##లనూ'  : 0.1385079324245453
[MASK]: 'тех'  : 0.05093983933329582
[MASK]: 'tartozo'  : 0.02629460208117962
##లనూاساس [MASK] است. 
[MASK]: '##లనూ'  : 0.09942671656608582
[MASK]: 'wade'  : 0.03139755129814148
[MASK]: 'тех'  : 0.029888354241847992
اصل اصلا انسان انسانوار
اصلا [MASK] اصل است.
[MASK]: '##లనూ'  : 0.07324353605508804
[MASK]: 'тех'  : 0.0503210686147213
[MASK]: 'filles'  : 0.032479435205459595
##లనూانسان [MASK] است. 
[MASK]: '##లనూ'  : 0.0649094358086586
[MASK]: 'тех'  : 0.042749930173158646
[MASK]: 'tartozo'  : 0.028681352734565735
عمل عملا اجبار اجبارا
عملا [MASK] عمل است.
[MASK]: '##లనూ'  : 0.06539096683263779
[MASK]: 'sp

In [57]:
Accuracy= 0
y= df[df["cat"]=="gram_antonym"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_antonym"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["first"][i[0]] + " و " + df["second"][i[0]]+"[MASK]" + " یکدیگر هستند."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10)  

خشنود ناخشنود موفق ناموفق
خشنود و ناخشنود[MASK] یکدیگر هستند.
[MASK]: '##றறில'  : 0.059871360659599304
[MASK]: '##లనూ'  : 0.043941814452409744
[MASK]: 'тех'  : 0.03565691411495209
##றறிலموفق [MASK] است. 
[MASK]: 'filles'  : 0.05598613619804382
[MASK]: 'тех'  : 0.03298085927963257
[MASK]: '##లనూ'  : 0.03046492300927639
درست نادرست پخته ناپخته
درست و نادرست[MASK] یکدیگر هستند.
[MASK]: 'тех'  : 0.04875215142965317
[MASK]: 'filles'  : 0.03077719919383526
[MASK]: 'tartozo'  : 0.0254979208111763
техپخته [MASK] است. 
[MASK]: '##లనూ'  : 0.06278179585933685
[MASK]: 'формула'  : 0.03147992119193077
[MASK]: 'tartozo'  : 0.028127122670412064
مناسب نامناسب همزمان ناهمزمان
مناسب و نامناسب[MASK] یکدیگر هستند.
[MASK]: '##లనూ'  : 0.04662482067942619
[MASK]: 'wade'  : 0.026768358424305916
[MASK]: 'тех'  : 0.025169575586915016
##లనూهمزمان [MASK] است. 
[MASK]: '##లనూ'  : 0.08958222717046738
[MASK]: 'descenso'  : 0.04697155952453613
[MASK]: 'tartozo'  : 0.03679267317056656
مربوط نامربوط منسجم نامنسجم
مربوط

In [58]:
Accuracy= 0
y= df[df["cat"]=="gram_comparative"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_comparative"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+ "[MASK]" + " "   +  df["third"][i[0]] + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

سریع سریعتر بلند بلندتر
سریعتر [MASK] سریع است.
[MASK]: '##లనూ'  : 0.11743944138288498
[MASK]: 'tartozo'  : 0.0485696904361248
[MASK]: '##றறில'  : 0.033682987093925476
##లనూ[MASK] بلند است. 
[MASK]: '##లనూ'  : 0.06632734835147858
[MASK]: 'тех'  : 0.03722742572426796
[MASK]: 'wade'  : 0.03187226131558418
بزرگ بزرگتر شدید شدیدتر
بزرگتر [MASK] بزرگ است.
[MASK]: 'spolecnost'  : 0.049683019518852234
[MASK]: 'wade'  : 0.049039892852306366
[MASK]: 'тех'  : 0.035505905747413635
spolecnost[MASK] شدید است. 
[MASK]: 'тех'  : 0.04977836459875107
[MASK]: 'формула'  : 0.036728423088788986
[MASK]: '##లనూ'  : 0.0356004498898983
مهم مهمتر بارز بارزتر
مهمتر [MASK] مهم است.
[MASK]: 'tartozo'  : 0.04406945779919624
[MASK]: '##లనూ'  : 0.043211184442043304
[MASK]: 'тех'  : 0.030726762488484383
tartozo[MASK] بارز است. 
[MASK]: '##లనూ'  : 0.12451576441526413
[MASK]: '##றறில'  : 0.027549980208277702
[MASK]: 'filles'  : 0.026996254920959473
به بهتر سرد سردتر
بهتر [MASK] به است.
[MASK]: '##లనూ'  : 0.208094149827

In [59]:
Accuracy= 0
y= df[df["cat"]=="gram_nationality"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_nationality"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+predicted+" "+ df["third"][i[0]] + " "   +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

اتریش اتریشی اسپانیا اسپانیایی
اتریشی [MASK] اتریش است.
[MASK]: '缽'  : 0.03173520043492317
[MASK]: '##లనూ'  : 0.027721725404262543
[MASK]: 'spolecnost'  : 0.025356436148285866
缽 اسپانیا [MASK] است. 
[MASK]: '##లనూ'  : 0.10771533846855164
[MASK]: 'tartozo'  : 0.03237200528383255
[MASK]: 'тех'  : 0.031533628702163696
سوئد سوئدی فنلاند فنلاندی
سوئدی [MASK] سوئد است.
[MASK]: '##లనూ'  : 0.050907645374536514
[MASK]: 'territoires'  : 0.04036692529916763
[MASK]: 'tartozo'  : 0.024746954441070557
##లనూ فنلاند [MASK] است. 
[MASK]: '缽'  : 0.05946541950106621
[MASK]: '##లనూ'  : 0.041217997670173645
[MASK]: 'spolecnost'  : 0.031198786571621895
بحرین بحرینی عراق عراقی
بحرینی [MASK] بحرین است.
[MASK]: '##లనూ'  : 0.07802017778158188
[MASK]: 'tartozo'  : 0.02973145991563797
[MASK]: 'тех'  : 0.024927426129579544
##లనూ عراق [MASK] است. 
[MASK]: '##లనూ'  : 0.06019122153520584
[MASK]: 'tartozo'  : 0.040568869560956955
[MASK]: 'формула'  : 0.03850460425019264
کویت کویتی سوریه سوری
کویتی [MASK] کویت است.
[MA

In [60]:
Accuracy= 0
y= df[df["cat"]=="gram_plural"]
for i in enumerate(df["cat"]):  
  #City
  if (i[1] == "gram_plural"):
    print(df["first"][i[0]] + " " + df["second"][i[0]] + " " + df["third"][i[0]] + " " + df["fourth"][i[0]])
    x = "" + df["second"][i[0]] + " " + "[MASK]" + " " +df["first"][i[0]] + " است."
    print (x)
    predicted = predict_masked_word(x,trained_bert)
    z = ""+ df["third"][i[0]] + " " + predicted  +  "[MASK]" + " است. "
    print(z)
    predicted1 = predict_masked_word(z,trained_bert)
    if (predicted1 == df["fourth"][i[0]]):
        Accuracy = Accuracy + 1   
num = y["cat"].count()      
print(Accuracy/10) 

تالیف تالیفات مخلوق مخلوقات
تالیفات [MASK] تالیف است.
[MASK]: '##లనూ'  : 0.08463925123214722
[MASK]: 'тех'  : 0.03321070224046707
[MASK]: '##юцца'  : 0.022547738626599312
مخلوق ##లనూ[MASK] است. 
[MASK]: 'тех'  : 0.13043789565563202
[MASK]: '##లనూ'  : 0.09837991744279861
[MASK]: 'descenso'  : 0.03390328213572502
اثر اثرات ارزش ارزشها
اثرات [MASK] اثر است.
[MASK]: '##றறில'  : 0.04524766653776169
[MASK]: '##లనూ'  : 0.038057539612054825
[MASK]: '缽'  : 0.02994597889482975
ارزش ##றறில[MASK] است. 
[MASK]: '##లనూ'  : 0.11366188526153564
[MASK]: 'filles'  : 0.01961878500878811
[MASK]: 'тех'  : 0.015233583748340607
انقلابی انقلابیون مورد موارد
انقلابیون [MASK] انقلابی است.
[MASK]: '##లనూ'  : 0.11457005888223648
[MASK]: 'тех'  : 0.078410804271698
[MASK]: 'tartozo'  : 0.04827092960476875
مورد ##లనూ[MASK] است. 
[MASK]: 'формула'  : 0.055143166333436966
[MASK]: '##లనూ'  : 0.03422778099775314
[MASK]: '##тің'  : 0.03044905886054039
حواری حواریون مورد موارد
حواریون [MASK] حواری است.
[MASK]: 'тех'  : 0.